## Lecture 10. GAN
Fashion MNIST 데이터를 처리하는 GAN 모델을 만들어 보자

Step 0. 이것저것 준비

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import random
import numpy as np

from utilities import EarlyStopping

In [2]:
L_R = 1e-3
BATCH_SIZE = 100
EPOCHS = 200

In [3]:
if torch.cuda.is_available()==True:
  DEVICE='cuda'
else:
  DEVICE='cpu'
np.random.seed(0b011011)
random.seed(0b011011)
torch.manual_seed(0b011011)
if DEVICE == 'cuda':
  torch.cuda.manual_seed_all(0b011011)



Step 1. Data

In [20]:
train_data = datasets.FashionMNIST(
    root="F:\Python_Codes\Data_for_Practice", 
    train=True, 
    download=True, 
    transform=ToTensor()
    )



test_data= datasets.FashionMNIST(
    root="F:\Python_Codes\Data_for_Practice", 
    train=False, 
    download=True, 
    transform=ToTensor()
    )


In [19]:
train_loader =DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader =DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

Step 2. Model

In [9]:
class Generator(nn.Module):
    def __init__(self, latent_dim=100, img_shape=(28,28)):
        self.latent_dim=latent_dim
        self.img_shape=img_shape
        super(Generator,self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers=[nn.Linear(in_feat,out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat))
            layers.append(nn.ReLU())
            return layers

        self.model= nn.Sequential(
            *block(self.latent_dim,128, normalize=False),
            *block(128,256),
            *block(256,512),
            *block(512,1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )
    
    def forward(self,z):
        img=self.model(z)
        img=img.view(img.size(0),*img_shape)
        return img

In [10]:
class Discriminator(nn.Module):
    def __init__(self, img_shape=(28,28)):
        super(Discriminator,self).__init__()
    
        self.model=nn.Sequential(
            nn.Linear(int(np.prod(img_shape)),512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256,1),
            nn.Sigmoid()
        )

    def forward(self,img):
        img=img.view(img.size(0),-1)
        out=self.model(img)


In [11]:
gen=Generator().to(DEVICE)
dis=Discriminator().to(DEVICE)

Step 3 and 4. Loss and Optimization

In [12]:
advloss_fn=nn.BCELoss().to(DEVICE)

G_opt = torch.optim.Adam(gen.parameters(), lr=L_R)
D_opt = torch.optim.Adam(dis.parameters(), lr=L_R)

Training

In [15]:
def train_loop(dataloader, generator, discriminator, advloss_fn, G_opt, D_opt):
    gen_avgloss=0
    dis_avgloss=0
    for i, (imgs,_) in enumerate(dataloader):
        imgs=imgs.to(DEVICE)
        valid=torch.Tensor(imgs.size(0),1, requires_grad=False).fill_(1.0).to(DEVICE)
        fake=torch.Tensor(imgs.size(0),0, requires_grad=False).fill_(0.0).to(DEVICE)

        #Train Generator
        G_opt.zero_grad()

        z=Tensor(np.random.normal(0,1,(imgs.shape[0], 100))).to(DEVICE)
        gen_imgs=generator(z)
        
        g_loss=advloss_fn(discriminator(gen_img),valid)
        gen_avgloss+=g_loss
        g_loss.backward()
        G_opt.step()
       
        #Train Discriminator
        real_loss=advloss_fn(discriminator(real_imgs), valid)
        fake_loss=advloss_fn(discriminator(gen_imgs.detach()),fake)

        d_loss=(real_loss+fake_loss)/2
        dis_avgloss+=d_loss
        d_loss.backward()
        D_opt.step()
    return gen_avgloss.item()/len(dataloader), dis_avgloss.item()/len(dataloader)

In [16]:
for epoch in range(EPOCHS):
    gen_avgloss, dis_avgloss=train_loop(train_loader, Generator, Discriminator, advloss_fn, G_opt, D_opt)
    print(
            "[Epoch %d/%d]  [D loss: %f] [G loss: %f]"
            % (epoch, EPOCHS, dis_avgloss.item(), gen_avgloss.item())
        )
    

TypeError: new() received an invalid combination of arguments - got (int, int, requires_grad=bool), but expected one of:
 * (*, torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
 * (object data, *, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
